# SEM IP Test example

## Block Diagram

![Figure 1. Block Diagram](img/diagram.png)

Here we can see the diagram for this bitstream. It consists in: AXI GPIO, AXI Uartlite and SEM IP.

### SEM_IP

1. The sem_ip module can communicate with the ZYNQ using the AXI Uartlite block.
2. The status bits are showed in PYNQ-Z2 board leds.
   * led[0] = SEM Injection status
   * led[1] = SEM Initialization status
   * led[2] = SEM ICAP Enable
   
### AXI_GPIO

Controls the ICAP_GRANT signal from SEM_IP. If I put a logic 1. SEM_IP will start.

### AXI_UARTLITE

Controls the communication between SEM_IP and PS.

## ICAP and PCAP

During boot of the Zynq-7000 Processing System (PS), access to the configuration logic in the device is given to the PS through the processor configuration access port (PCAP). This provides a path for the PS bootloader to download a bitstream to the Zynq-7000 Programmable Logic (PL). When the PS bootloader completes, the PS and PCAP remain in control of the configuration logic to support partial reconfiguration of the PL by the PS. 

However, while the PS and PCAP are in control of the configuration logic, the PL and ICAP are locked out of the configuration logic. In order for the controller to function, configuration logic access must be transferred to the ICAP. This is accomplished by clearing PCAP_PR (bit 27) in the PS device configuration control register (DEVCFG CTRL, address 0xF8007000).

In [1]:
# Import the PS library
from pynq import ps

# Create a instance of DEV CFG 
XDCFG_BASE_ADDRESS = 0xF8007000
dev_cfg_ctrl = ps.Register(XDCFG_BASE_ADDRESS)

![Figure 2. ICAP Controller](img/pcap.png)

## Import libraries and load the overlay

In [2]:
# Import Libraries
from pynq import Overlay
from pynq.lib import AxiGPIO
import sys
sys.path.insert(1, './src')
from uartlite import *

# Download the overlay
ol = Overlay("./bits/sem_ip.bit")
ol.download()

## Initialize AXI GPIO 

In [3]:
# SEM Enable GPIO Offset
SEM_EN_OFFSET = 0
# SEM Enable GPIO handle, the sem IP enable is the offset number 0
sem_enable = ol.sem_en_gpio

## Initialize AXI Uartlite

In [4]:
# Address of the ip core
ADDRESS = 0x42c00000  
uart = UartAXI(ADDRESS)

# Setup AXI UART register
uart.setupCtrlReg()

## Clean the PCAP_PR bit from DEVCFG CTRL register

I have to clean this bit to use ICAP instead of PCAP

In [5]:
# Clear PCAP_PR bit
dev_cfg_ctrl[27] = 0

if dev_cfg_ctrl[27] == 0:
    print("Info: ICAP Enabled")
else:
    print("Warning: ICAP not Enabled")

Info: ICAP Enabled


## Initialize SEM IP

In [6]:
# Enable ICAP Grant signal
sem_enable.write(SEM_EN_OFFSET,1)

# Print Init Report
print("Init Report:")
print("------------")
print("")
for x in range(7):
    print(uart.readLine())
print(uart.read(3))

Init Report:
------------

X7_SEM_V4_1
SC 01
FS 0E
ICAP OK
RDBK OK
INIT OK
SC 02
O> 
